<a href="https://colab.research.google.com/github/Abdelrhman129/my-_colab_notbook/blob/main/pert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
texts = [
    "I love this movie!",
    "This film is terrible.",
    "Amazing acting and great story.",
    "I hate this movie so much.",
    "What a fantastic experience!"
]
labels = [1, 0, 1, 0, 1]

In [3]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

#load Bert tokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
def encode_texts(text_list, max_len=32):
    encoding = tokenizer(
        text_list,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='tf'
    )

    return encoding['input_ids'], encoding['attention_mask']

In [6]:
train_inputs, train_masks = encode_texts(train_texts)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [7]:
test_inputs, test_masks = encode_texts(test_texts)

In [8]:
train_inputs

<tf.Tensor: shape=(4, 32), dtype=int32, numpy=
array([[  101,  2054,  1037, 10392,  3325,   999,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [  101,  6429,  3772,  1998,  2307,  2466,  1012,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [  101,  1045,  2293,  2023,  3185,   999,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [  101,  1045,  5223,  2023,  3185,  2061,  2172,  1012,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     

In [9]:
test_inputs

<tf.Tensor: shape=(1, 32), dtype=int32, numpy=
array([[ 101, 2023, 2143, 2003, 6659, 1012,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)>

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_inputs, "attention_mask": train_masks},
    train_labels
)).shuffle(len(train_labels)).batch(2)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": test_inputs, "attention_mask": test_masks},
    test_labels
)).batch(2)


In [11]:
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    use_safetensors=False
)

tf_model.h5:   0%|          | 0.00/536M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [13]:
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

Epoch 1/3
2/2 [==============================] - 103s 13s/step - loss: 0.6842 - accuracy: 0.5000 - val_loss: 0.7519 - val_accuracy: 0.0000e+00
Epoch 2/3
2/2 [==============================] - 13s 7s/step - loss: 0.5480 - accuracy: 1.0000 - val_loss: 0.8397 - val_accuracy: 0.0000e+00
Epoch 3/3
2/2 [==============================] - 9s 5s/step - loss: 0.5380 - accuracy: 1.0000 - val_loss: 0.9228 - val_accuracy: 0.0000e+00


In [14]:
preds = model.predict(test_dataset).logits
pred_labels = np.argmax(preds, axis=1)
accuracy = accuracy_score(test_labels, pred_labels)

1/1 [==============================] - 8s 8s/step


In [15]:
print(accuracy)

0.0
